In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import KFold
from sklearn import linear_model, metrics, preprocessing
from sklearn.model_selection import KFold, train_test_split
import xgboost

import azureml.core
from azureml.core import Dataset
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core import Experiment, Workspace

import sys
sys.path.append('../')
sys.path.append('../scripts/')

In [2]:
# Log In to Azure ML Workspace
interactive_auth = InteractiveLoginAuthentication(tenant_id="76f90eb1-fb9a-4446-9875-4d323d6455ad")

# Initialise workspace
ws = Workspace.from_config(auth=interactive_auth)

# Data import 
aml_dataset = Dataset.get_by_name(ws, 'train_ds', version='latest')
data = aml_dataset.to_pandas_dataframe()
data.set_index('CardHolder', inplace=True)

# Save it to the further opeartions
original_columns = data.columns
data_origin = data.copy()

/anaconda/envs/py37_default/lib/python3.7/site-packages/azureml/dataprep/api/dataflow.py:722: UserWarning: Your pandas and pyarrow versions are incompatible. Please install pyarrow>=0.12.0 for improved performance of to_pandas_dataframe. You can ensure the correct version is installed by running: pip install pyarrow>=0.12.0 --upgrade
  warnings.warn('Your pandas and pyarrow versions are incompatible. '


In [3]:
from importlib import reload
import features.functions
reload(features.functions)

data = features.functions.preprocessing(data)
data_w_features = data.copy()

/anaconda/envs/py37_default/lib/python3.7/site-packages/pandas/core/generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
/anaconda/envs/py37_default/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [5]:
from scripts.metrics import custom_metric
import features.functions
reload(features.functions)

data = data_w_features.copy()
data = data.reset_index().drop('CardHolder', axis=1)
data.loc[:, 'group'].replace({1 : 'test', 0 : 'control'}, inplace=True)

data['class'] = 0
data.loc[(data['group'] == 'control') & (data['response_att'] == 1), 'class'] = 1
data.loc[(data['group'] == 'test') & (data['response_att'] == 0), 'class'] = 2
data.loc[(data['group'] == 'test') & (data['response_att'] == 1), 'class'] = 3

train, test = train_test_split(data)
train, test = features.functions.feature_generation(train, test, 
                                mean_columns=['group', 'response_att', 'stdev_days_between_visits_15d'],
                                mean_index_columns=['gender', 'age', 'children'])

x_train = train.drop(['class', 'group', 'response_att'], axis=1)
y_train = train['class']

x_test = test.drop(['class', 'group', 'response_att'], axis=1)
y_test = test['class']

params = {'n_estimators': 20, 'eval_metric': ['merror', 'mlogloss'], 'verbose': True, 'n_jobs': -1}
model = xgboost.XGBClassifier(**params)

model.fit(x_train, y_train, verbose=True, eval_set=[(x_test, y_test)])

pred = model.predict_proba(x_test)
final = pred[:,3] + pred[:,0] - pred[:,1] - pred[:,2]
test['uplift'] = final

custom_metric(test)

[0]	validation_0-merror:0.297526	validation_0-mlogloss:1.30815
[1]	validation_0-merror:0.297504	validation_0-mlogloss:1.24296
[2]	validation_0-merror:0.296513	validation_0-mlogloss:1.18769
[3]	validation_0-merror:0.296347	validation_0-mlogloss:1.14044
[4]	validation_0-merror:0.296287	validation_0-mlogloss:1.09971
[5]	validation_0-merror:0.296287	validation_0-mlogloss:1.06446
[6]	validation_0-merror:0.296302	validation_0-mlogloss:1.0339
[7]	validation_0-merror:0.296332	validation_0-mlogloss:1.00715
[8]	validation_0-merror:0.296295	validation_0-mlogloss:0.983543
[9]	validation_0-merror:0.29622	validation_0-mlogloss:0.962832
[10]	validation_0-merror:0.296235	validation_0-mlogloss:0.944663
[11]	validation_0-merror:0.296362	validation_0-mlogloss:0.928456
[12]	validation_0-merror:0.296408	validation_0-mlogloss:0.914127
[13]	validation_0-merror:0.296212	validation_0-mlogloss:0.901299
[14]	validation_0-merror:0.296227	validation_0-mlogloss:0.890002
[15]	validation_0-merror:0.296145	validation_

3.111861766198451